In [28]:
# Setup: Add project root to path and import utilities
import sys; sys.path.insert(0, '..')
from utils import get_input

# Load input data for day 8
data = get_input(8)

print(f"Loaded {len(data)} lines of input")
for line in data:
    print(line)

Loaded 1000 lines of input
27558,61383,12726
15513,81970,25554
24379,89821,82524
42987,15460,38773
10680,2978,15903
77950,19916,8194
84115,76354,95701
75128,80898,64571
71561,94114,61650
41769,64801,84116
99451,81585,8703
57564,2417,64836
84589,38412,15385
35597,52203,16486
42032,73225,53406
38913,2543,47366
18003,27047,63249
15725,35221,8917
67459,89481,6379
93053,35768,21677
57854,1647,34547
26887,45673,77882
73271,62199,8639
84107,45639,50143
43769,90773,81919
3151,61180,99236
6927,79733,83957
49142,71115,17212
57941,89686,33277
11148,25706,76435
59562,53048,55156
26033,72414,65197
35437,81135,37478
43184,8900,19814
39591,5679,77570
76644,39268,69061
44173,17521,57147
87465,41898,44085
40053,91676,2430
65503,72760,2043
69780,10458,14163
99262,7255,7737
25197,94672,68707
13113,84009,19685
22729,28488,37015
37615,72443,48207
43754,94508,4531
80141,39964,57206
12264,55521,63720
14183,59921,42797
80687,29108,13747
75010,68740,69291
39507,46719,11879
75662,23096,21415
36451,81573,81035
5

In [29]:
# make a dataclass of data with X,Y,Z the three values in each line split by commas
from dataclasses import dataclass
from typing import List
@dataclass
class JunctionBox:
    X: int
    Y: int
    Z: int
    FK_circuit: int = None # foreign key to Circuit, none by default
junction_boxes: List[JunctionBox] = []
for line in data:
    x, y, z = map(int, line.split(','))
    junction_boxes.append(JunctionBox(X=x, Y=y, Z=z))
print(junction_boxes)

[JunctionBox(X=27558, Y=61383, Z=12726, FK_circuit=None), JunctionBox(X=15513, Y=81970, Z=25554, FK_circuit=None), JunctionBox(X=24379, Y=89821, Z=82524, FK_circuit=None), JunctionBox(X=42987, Y=15460, Z=38773, FK_circuit=None), JunctionBox(X=10680, Y=2978, Z=15903, FK_circuit=None), JunctionBox(X=77950, Y=19916, Z=8194, FK_circuit=None), JunctionBox(X=84115, Y=76354, Z=95701, FK_circuit=None), JunctionBox(X=75128, Y=80898, Z=64571, FK_circuit=None), JunctionBox(X=71561, Y=94114, Z=61650, FK_circuit=None), JunctionBox(X=41769, Y=64801, Z=84116, FK_circuit=None), JunctionBox(X=99451, Y=81585, Z=8703, FK_circuit=None), JunctionBox(X=57564, Y=2417, Z=64836, FK_circuit=None), JunctionBox(X=84589, Y=38412, Z=15385, FK_circuit=None), JunctionBox(X=35597, Y=52203, Z=16486, FK_circuit=None), JunctionBox(X=42032, Y=73225, Z=53406, FK_circuit=None), JunctionBox(X=38913, Y=2543, Z=47366, FK_circuit=None), JunctionBox(X=18003, Y=27047, Z=63249, FK_circuit=None), JunctionBox(X=15725, Y=35221, Z=891

In [30]:
def distance(box1: JunctionBox, box2: JunctionBox) -> int:
    # use euclidean distance
    return ((box1.X - box2.X) ** 2 + (box1.Y - box2.Y) ** 2 + (box1.Z - box2.Z) ** 2) ** 0.5

# create dataclass of connections between all junction boxes
@dataclass
class Connection:
    box1: JunctionBox
    box2: JunctionBox
    distance: int
connections: List[Connection] = []
for i in range(len(junction_boxes)):
    for j in range(i + 1, len(junction_boxes)):
        dist = distance(junction_boxes[i], junction_boxes[j])
        connections.append(Connection(box1=junction_boxes[i], box2=junction_boxes[j], distance=dist))


In [31]:
# show some information about connections
print(f"Total connections: {len(connections)}")
max_connection = max(connections, key=lambda c: c.distance)
print(f"Max connection: {max_connection}")
# show a sample of connections
print("Sample connections:")
for conn in connections[:5]:
    print(conn)

Total connections: 499500
Max connection: Connection(box1=JunctionBox(X=1877, Y=92653, Z=150, FK_circuit=None), box2=JunctionBox(X=90615, Y=818, Z=99061, FK_circuit=None), distance=161528.59124625585)
Sample connections:
Connection(box1=JunctionBox(X=27558, Y=61383, Z=12726, FK_circuit=None), box2=JunctionBox(X=15513, Y=81970, Z=25554, FK_circuit=None), distance=27082.543787465755)
Connection(box1=JunctionBox(X=27558, Y=61383, Z=12726, FK_circuit=None), box2=JunctionBox(X=24379, Y=89821, Z=82524, FK_circuit=None), distance=75435.9774179403)
Connection(box1=JunctionBox(X=27558, Y=61383, Z=12726, FK_circuit=None), box2=JunctionBox(X=42987, Y=15460, Z=38773, FK_circuit=None), distance=55003.83785700776)
Connection(box1=JunctionBox(X=27558, Y=61383, Z=12726, FK_circuit=None), box2=JunctionBox(X=10680, Y=2978, Z=15903, FK_circuit=None), distance=60877.781152075506)
Connection(box1=JunctionBox(X=27558, Y=61383, Z=12726, FK_circuit=None), box2=JunctionBox(X=77950, Y=19916, Z=8194, FK_circuit=

In [32]:
# define circuit, a set of JunctionBoxes
@dataclass
class Circuit:
    junction_boxes: List[JunctionBox]
    primary_key: int
    def __post_init__(self):
        self.size = len(self.junction_boxes)
        for box in self.junction_boxes:
            box.FK_circuit = self.primary_key
    def append(self, box: JunctionBox):
        if box not in self.junction_boxes:
            self.junction_boxes.append(box)
            box.FK_circuit = self.primary_key
            self.size = len(self.junction_boxes)
    def remove(self, box: JunctionBox):
        if box in self.junction_boxes:
            self.junction_boxes.remove(box)
            box.FK_circuit = None
            self.size = len(self.junction_boxes)
    def move(self, box: JunctionBox, target_circuit: 'Circuit'):
        self.remove(box)
        target_circuit.append(box)
    

In [33]:
# define a grid class to hold junction boxes and connections
@dataclass
class Grid:
    junction_boxes: List[JunctionBox]
    connections: List[Connection]
    circuits: List[Circuit] = None
    
    def __post_init__(self):
        self.circuits = []
        self._next_circuit_pk = 0  # Initialize counter AFTER dataclass init
        self.nr_of_boxes = len(self.junction_boxes)

    def get_circuit(self, circuit_pk: int) -> Circuit:
        for circuit in self.circuits:
            if circuit.primary_key == circuit_pk:
                return circuit
        return None
    
    def is_complete(self) -> bool:
        # check if all junction boxes are in a circuit
        # this is equal to the number of junction boxes being equal to the sum of junction boxes in all circuits
        total_boxes_in_circuits = sum(circuit.size for circuit in self.circuits)
        return total_boxes_in_circuits == self.nr_of_boxes

    def activate_connection(self, connection: Connection):
        # update circuits
        # if neither connection boxes are in any circuit, create a new circuit
        # if both boxes are in different circuits, merge the circuits
        # if one box is in a circuit, add the other box to that circuit
        
        # find circuits containing box1 and box2 by foreign key
        box1 = connection.box1
        circuit1 = self.get_circuit(box1.FK_circuit)
        box2 = connection.box2
        circuit2 = self.get_circuit(box2.FK_circuit)
        
        if circuit1 is None and circuit2 is None:
            # Neither box is in a circuit - create new circuit
            circuit = Circuit(junction_boxes=[box1, box2], 
                            primary_key=self._next_circuit_pk)
            self._next_circuit_pk += 1
            self.circuits.append(circuit)
            
        elif circuit1 is None and circuit2 is not None:
            # Only box2 is in a circuit - add box1 to it
            circuit2.append(box1)
            
        elif circuit1 is not None and circuit2 is None:
            # Only box1 is in a circuit - add box2 to it
            circuit1.append(box2)
            
        elif circuit1 is not None and circuit2 is not None:
            # Both boxes are in circuits
            if circuit1.primary_key != circuit2.primary_key:
                # Different circuits - merge them           
                for box in list(circuit2.junction_boxes):  # Use list() to avoid modifying during iteration
                    circuit2.move(box, circuit1)
                self.circuits.remove(circuit2)


In [34]:
# Create deep copies of junction boxes and connections to avoid modifying the originals
from copy import deepcopy

# Deep copy both junction boxes and connections
grid_junction_boxes = deepcopy(junction_boxes)
grid_connections = deepcopy(connections)

In [35]:
grid = Grid(junction_boxes=grid_junction_boxes, connections=grid_connections)

In [36]:

# sort connections by distance
grid.connections.sort(key=lambda c: c.distance)
# activate shortest 1000 connections in grid
for connection in grid.connections[:1000]:
    grid.activate_connection(connection)


In [37]:
# take the product of the sizes of the three largest circuits
circuit_sizes = sorted([len(circuit.junction_boxes) for circuit in grid.circuits], reverse=True)
from math import prod
prod(circuit_sizes[:3])

52668

In [38]:
grid = Grid(junction_boxes=grid_junction_boxes, connections=grid_connections)

# continue activating connections until the grid is complete
# then save the X coordinates of the last activated connection
while not grid.is_complete():
    for connection in grid.connections:
        grid.activate_connection(connection)
        if grid.is_complete():
            last_activated_connection = connection
            break


print(last_activated_connection.box1.X * last_activated_connection.box2.X)

1474050600
